In [1]:
# Check on gpu
!nvidia-smi

Wed Oct  1 11:08:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.144                Driver Version: 570.144        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:CA:00.0 Off |                    0 |
|  0%   26C    P8             21W /  300W |       0MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from tifffile import imread, imwrite

from scipy.ndimage import gaussian_filter
from skimage.morphology import closing, disk

import os
from glob import glob
from tqdm import tqdm



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	linux 
python version: 	3.9.23 
torch version:  	2.7.1+cu126! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [3]:
# Initialize Cellpose model
import torch
if torch.cuda.is_available():
    gpu = True
else:
    gpu = False
model = models.CellposeModel(gpu=gpu)

In [4]:
# Sample segmentation
# Set the master folder path here
master_folder = '/home/vil945/live_cell_imaging/2025-09-30_live_cell_imaging'

# Recursively find folders with specified channel in their names
target_folders = []
for root, dirs, files in os.walk(master_folder):    
    for dir_name in dirs:
        if '_segmented' in dir_name:
            continue  # Skip _segmented folders themselves
        if '_mCherry' in dir_name or '_GFP' in dir_name or '_CTV' in dir_name:   
            target_folders.append(os.path.join(root, dir_name))

# Process each matching folder
for folder in tqdm(target_folders, desc="Processing target folders"):
    input_files = glob(os.path.join(folder, '*.tif'))
    if not input_files:
        continue  # Skip empty folders

    # Create output folder inside the current one
    folder_name = os.path.basename(folder)
    output_folder = os.path.join(folder, f'{folder_name}_segmented')
    os.makedirs(output_folder, exist_ok=True)

    for filepath in tqdm(input_files, desc=f"Segmenting in {folder_name}", leave=False):
        # Read multi-frame TIFF
        img_stack = imread(filepath)
        n_frames = img_stack.shape[0]

        label_stack = []

        # Segment each frame
        if "_PI" in folder_name: # Special case of PI channel (requires pre-processing)
            for i in range(n_frames):
                img_blur = gaussian_filter(img_stack[i], sigma=2) # Smoothing
                img_closed = closing(img_blur, disk(3)) # Morphological closing
                
                current_label = model.eval(img_closed, diameter=None, channels=[0,0], flow_threshold=0.3, cellprob_threshold=-3.0)[0]
                label_stack.append(current_label)

            # Save labeled multi-frame image
            label_stack = np.stack(label_stack).astype(np.uint16)
            filename = os.path.basename(filepath)
            save_path = os.path.join(output_folder, f'segmented_{filename}')
            imwrite(save_path, label_stack)

        else:
            for i in range(n_frames):
                current_label = model.eval(img_stack[i], diameter=None, channels=[0,0])[0]
                label_stack.append(current_label)
    
            # Save labeled multi-frame image
            label_stack = np.stack(label_stack).astype(np.uint16)
            filename = os.path.basename(filepath)
            save_path = os.path.join(output_folder, f'segmented_{filename}')
            imwrite(save_path, label_stack)

print("All segmentations complete.")

Segmenting in 2025-09-30_CTV:   0%|          | 0/2 [00:00<?, ?it/s]channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 channels, only the first 3 channels will be used
channels deprecated in v4.0.1+. If data contain more than 3 c

All segmentations complete.
